In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

In [3]:
# load data and split into training and test sets
train_df = pd.read_csv("fish_participant.csv")
test_df = pd.read_csv("fish_holdout_demo.csv")

In [10]:
train_df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
1,Perch,110.0,20.0,22.0,23.5,5.5225,3.9950
2,Roach,160.0,20.5,22.5,25.3,7.0334,3.8203
3,Parkki,60.0,14.3,15.5,17.4,6.5772,2.3142
4,Bream,700.0,30.4,33.0,38.3,14.8604,5.2854


In [9]:
train_df.shape

(111, 7)

In [15]:
x_train = train_df[["Length1", "Length2", "Length3", "Height", "Width"]]
x_test = train_df[["Weight"]]
y_train = train_df[["Length1", "Length2", "Length3", "Height", "Width"]]
y_test = train_df[["Weight"]]

In [16]:
# now we will construct and fit our model using gradient boost regression

regressor = GradientBoostingRegressor(
    max_depth=5,
    n_estimators=8,
    learning_rate=0.1)

regressor.fit(x_train, y_train)

ValueError: bad input shape (111, 5)

In [12]:
# get the optimal number of trees using the staged_predict()
errors = [mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(x_test)]

best_n_estimators = np.argmin(errors)

NameError: name 'regressor' is not defined

In [ ]:
# Fitting Score
print("")
print("___Gradient Boosting Machine Score__")
print(regressor.train_score_)